In [10]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()





### Extract CSVs into DataFrames

In [11]:
la_airbnb_file = "C:/Users/sinan/OneDrive/Desktop/ETL Project/Resources/NewOrleans_airbnb_listing.csv"
la_airbnb_df = pd.read_csv(la_airbnb_file)
la_airbnb_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,10291,https://www.airbnb.com/rooms/10291,20190306190816,43530,Spacious Cottage in Mid-City!,NaN,"Spacious house located in Mid-City, New Orlean...","Spacious house located in Mid-City, New Orlean...",none,NaN,...,f,f,moderate,t,f,1,1,0,0,0.99
1,19091,https://www.airbnb.com/rooms/19091,20190306190816,43530,Fully Furnished Cozy Apartment,CITY OF NEW ORLEANS STR LICENSE PERMIT NUMBER:...,SEE SPECIAL REDUCED SUMMER PRICING! This apa...,CITY OF NEW ORLEANS STR LICENSE PERMIT NUMBER:...,none,"NorthWest Carrollton, where this apartment is ...",...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,3.99
2,26834,https://www.airbnb.com/rooms/26834,20190306190816,43530,Maison Mandeville in the Marigny,NaN,Charming shotgun apartment in the Marigny neig...,Charming shotgun apartment in the Marigny neig...,none,NaN,...,f,f,strict_14_with_grace_period,f,t,2,2,0,0,2.10
3,53173,https://www.airbnb.com/rooms/53173,20190306190816,43530,MARDI GRAS YA’LL HUNDREDS 5 STAR REVIEWS !!!,Current articles in NYTimes and travel magazi...,This is a VERY large bedroom - boasting queen ...,Current articles in NYTimes and travel magazi...,none,"Funky creative types (sorta like Williamsburg,...",...,t,f,strict_14_with_grace_period,f,f,2,1,1,0,1.32
4,53544,https://www.airbnb.com/rooms/53544,20190306190816,43530,Historical Bywater Mother in Law Cottage,We are little slice of authentic NOLA living.....,We offering up a New Orleans experience. Our...,We are little slice of authentic NOLA living.....,none,Kentucky st not a very well known street in Ne...,...,f,f,strict_14_with_grace_period,f,f,2,1,1,0,1.76


In [12]:
la_hotels_file = "C:/Users/sinan/OneDrive/Desktop/ETL Project/Resources/NewOrleans_Hotels.csv"
la_hotels_df = pd.read_csv(la_hotels_file)
la_hotels_df.head()

,BusinessName,Address,Suite,City,State,Zip,BusinessType,OwnerName,X,Y,Location
0,CASA LA LICIOUS,1367 SAINT ANTHONY AVE,NaN,NEW ORLEANS,LA,70116,4703 - BED & BREAKFAST INNS,CASA LA LICIOUS,NaN,NaN,NaN
1,CASA PELICAN LLC,712 PELICAN ST,NaN,NEW ORLEANS,LA,70114,4703 - BED & BREAKFAST INNS,CASA PELICAN LLC,NaN,NaN,NaN
2,HAMPTON INN - ST. CHARLES,3626 SAINT CHARLES AVE,NaN,NEW ORLEANS,LA,70115,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"BRE NOLA PROPERTY OWNER, LLC",NaN,NaN,NaN
3,HISTORIC STREETCAR INN,1509 SAINT CHARLES AVE,NaN,NEW ORLEANS,LA,70130,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"1509 ST. CHARLES AVE, LLC",NaN,NaN,NaN
4,JELLY ROLL SUITES,320 ANDREW HIGGINS ST,NaN,NEW ORLEANS,LA,70130,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"VENTURE CCR OF N O,LLC",NaN,NaN,NaN


### Transform Airbnb  DataFrame

In [13]:
# Create a filtered dataframe from specific columns
la_airbnb_cols = ["id", "city", "state", "property_type", "room_type", "accommodates", "price", "minimum_nights", "zipcode"]
la_airbnb_transformed= la_airbnb_df[la_airbnb_cols].copy()


# Clean the data by dropping duplicates and setting the index
la_airbnb_transformed.drop_duplicates("id", inplace=True)
la_airbnb_transformed.set_index("id", inplace=True)

la_airbnb_transformed.head()

,city,state,property_type,room_type,accommodates,price,minimum_nights,zipcode
id,,,,,,,,
10291,New Orleans,LA,Cottage,Entire home/apt,4,300,2,70124
19091,New Orleans,LA,Apartment,Entire home/apt,3,100,1,70118
26834,New Orleans,LA,House,Entire home/apt,2,125,4,70117
53173,New Orleans,LA,House,Private room,2,115,3,70117
53544,New Orleans,LA,House,Entire home/apt,2,50,5,70117


### Transform Hotels DataFrame

In [14]:
la_hotels_cols = ["BusinessName", "Address", "City", "State", "Zip", "BusinessType", "OwnerName"]
la_hotels_transformed = la_hotels_df[la_hotels_cols].copy()

# Rename the column headers
la_hotels_transformed = la_hotels_transformed.rename(columns={"BusinessName": "businessname",
                                                         "Address": "address",
                                                         "City": "city",
                                                         "State": "state", "Zip":"zip", "BusinessType":"business_type", "OwnerName":"owner_name"})

# Set index
la_hotels_transformed.set_index("businessname", inplace=True)

la_hotels_transformed.head()

,address,city,state,zip,business_type,owner_name
businessname,,,,,,
CASA LA LICIOUS,1367 SAINT ANTHONY AVE,NEW ORLEANS,LA,70116,4703 - BED & BREAKFAST INNS,CASA LA LICIOUS
CASA PELICAN LLC,712 PELICAN ST,NEW ORLEANS,LA,70114,4703 - BED & BREAKFAST INNS,CASA PELICAN LLC
HAMPTON INN - ST. CHARLES,3626 SAINT CHARLES AVE,NEW ORLEANS,LA,70115,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"BRE NOLA PROPERTY OWNER, LLC"
HISTORIC STREETCAR INN,1509 SAINT CHARLES AVE,NEW ORLEANS,LA,70130,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"1509 ST. CHARLES AVE, LLC"
JELLY ROLL SUITES,320 ANDREW HIGGINS ST,NEW ORLEANS,LA,70130,4701 - HOTELS (EXC CASINO HOTELS) & MOTELS,"VENTURE CCR OF N O,LLC"


### Create database connection

In [15]:
connection_string = "root:hSinan17@localhost/etl"
engine = create_engine(f'mysql://{connection_string}')

In [16]:
# Confirm tables
engine.table_names()

['la_airbnb', 'la_hotels']

### Load DataFrames into database

In [17]:
la_airbnb_transformed.to_sql(name='la_airbnb', con=engine, if_exists='replace', index=True)

In [18]:
la_hotels_transformed.to_sql(name='la_hotels', con=engine, if_exists='append', index=False)